In [1]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import pickle

2024-01-05 11:56:21.509441: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 11:56:21.571687: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 11:56:21.572914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 11:56:25.719585: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def to_rf(df):
    df = df.fillna(-1)
    for col in df.columns:
        if col in ['dem','smap','precip','in_situ']:
            df[col] = df[col].astype('float32')
        elif col =='date':
            continue
        else:
            df[col] = df[col].astype('int16')
    return df

In [4]:
all_df = pd.read_pickle('Datasets/all_df_final.pkl')
test_df = pd.read_pickle('Datasets/val_df.pkl')
cond = all_df.index.isin(test_df.index)
train_df = all_df[~cond]
train_df = to_rf(train_df)

Tuners

In [ ]:
tuner_rf = tfdf.tuner.RandomSearch(num_trials=100)
tuner_rf.choice("min_examples", [2, 5, 7, 10])
tuner_rf.choice("num_trees", [100])
local_search_space = tuner_rf.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [8,16, 24, 32])
global_search_space = tuner_rf.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

## Single Ensemble Training 

In [ ]:
label = 'in_situ'
variables = ['sand','clay','texture','ph','dem','ET','NDVI_500','LST_11','precip','smap','in_situ']

for text in train_df.texture.unique():
    sub_df = train_df.loc[lambda d: d.texture == text].loc[:,variables].drop('texture',axis=1)
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(sub_df, label=label, task=tfdf.keras.Task.REGRESSION)
    model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION,verbose=0,
                                             tuner=tuner_rf)
    model.fit(x=train_ds)
    model.save(f'Models/RF/{text}',overwrite=True)
    with open(f'Models/RF/summaries/{text}.txt', 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))

## Crossfold Ensemble Training

In [ ]:
label = 'in_situ'
variables = ['sand','clay','texture','ph','dem','ET','NDVI_500','LST_11','precip','smap','in_situ']

for k in range(10):
    df = pd.read_csv(f'Datasets/Crossfold_Datasets/{k}.csv').set_index('Unnamed: 0')
    train_df = all_df.loc[df.index.unique()].loc[:,variables]
    train_df = to_rf(train_df)
    for text in train_df['texture'].unique():
        sub_df = train_df.loc[lambda d: d.texture == text].drop('texture',axis=1)
        train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(sub_df, label=label, task=tfdf.keras.Task.REGRESSION)
        model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION,verbose=0,
                                                 tuner=tuner_rf)
        model.fit(x=train_ds)
        model.save(f'Models/crossfold/rf/{k}/{text}',overwrite=True)
        with open(f'Models/crossfold/rf/{k}/summaries/{text}.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))